# Practical 2 

* Ron Ramsay's solutions to a practical problem set given within courses STAT2203/STAT7203 at the University of Queensland in Semester 2, 2020.
* A generalistic and programmatic approach is taken using Python 3.8.

In [1]:
import itertools
from math import factorial
from typing import Iterable

import numpy as np

## Question 1

Warren Buffet challenges Bill Gates to a game of dice. Gates picks one of the
following four sided dice, then Buffet chooses one of the remaining dice. Whoever
rolls the highest number wins. The three dice have the following numbers on their
faces:

    (die A): 12, 10, 3, 1
    (die B): 9, 8, 7, 2
    (die C): 11, 6, 5, 4
  
What is the probability of Buffett winning the game, assuming he chooses his die
to maximise his probability of winning?

##### Solution:

Let's define the problem's objects:

In [2]:
dice = {
    'A': (12, 10, 3, 1),
    'B': (9, 8, 7, 2),
    'C': (11, 6, 5, 4)}

**Which dice will the players choose?** Between two dice, first consider all the permutations of one roll each, and determine the proportion of rolls that win (i.e. have the higher rolled number) for a player; that will be the expectation of winning.

e.g. Such permutations for dice A and B are:

In [3]:
print(list(itertools.product(dice['A'], dice['B'])))

[(12, 9), (12, 8), (12, 7), (12, 2), (10, 9), (10, 8), (10, 7), (10, 2), (3, 9), (3, 8), (3, 7), (3, 2), (1, 9), (1, 8), (1, 7), (1, 2)]


The condition of the first die outcome beating the second, for each of those rolls, are:

In [4]:
print([a > b for (a, b) in itertools.product(dice['A'], dice['B'])])

[True, True, True, True, True, True, True, True, False, False, False, True, False, False, False, False]


The expectation of winning is the mean of those outcomes:

In [5]:
np.array([a > b for (a, b) in itertools.product(dice['A'], dice['B'])]).mean()

0.5625

Consolidating this process into a callable function, **the expectation of one dice beating another** is:

In [6]:
def two_dice_expectation(A: Iterable, B: Iterable) -> float:
    "return the expectation that dice `A` beats `B`."
    return np.array([a > b for (a, b) in itertools.product(A, B)]).mean()

In our problem, let's see what the **expectation of winnings** are between **all pair combinations of dice**:

In [7]:
for pair_keys in itertools.combinations(dice.keys(), r=2):
    expectation = two_dice_expectation(*[dice[key] for key in pair_keys])
    if expectation >= 0.5:
        print(pair_keys, expectation)
    else:
        print(pair_keys[::-1], 1 - expectation) # reverse the keys; take the complement

('A', 'B') 0.5625
('C', 'A') 0.5625
('B', 'C') 0.5625


Examining those results, we see a **cyclical situation** whereby: A beats B, B beats C, and C beats A,  --all at the same expectation of 56.25%.

Whichever die Bill Gates (nominated as the first player) picks, Warren Buffet will then choose the die that beats it. The **expectation of Buffett winning the game is 0.5625**.

The interesting thing about this game (with the set of dice given) is that: the player who is given the first choice of die is actually at a disadvantage.

_My extension question:_ 
Consider how the choice of die might be made for other sets of dice.

In [8]:
# (Tidy up to remove objects no longer needed.)
del dice, two_dice_expectation

## Question 2
We draw at random 5 numbers from $\left \{1, ..., 100 \right \}$, *with replacement* 
(e.g. drawing number 9 twice is possible). 
What is the probability that exactly 3 numbers are even?

##### Solution:

The probability that a single draw is an even number is $\frac{50}{100} = \frac{1}{2}$, so let's take advantage of the Equilikely Principle. We will consider the ratio of the number of possible outcomes in our event to that of the complete state space.

Let $A$ be our event, (whereby an outcome has an even number for exactly 3 of the 5 draws).

The number of outcomes in our event is: $|A| = {^{5}\textrm{C}_{3}} = \binom{5}{3} = \frac{5!}{3!(5-3)!} = \frac{5 \times 4}{2 \times 1}= 10$. 

>This is equivalent to the _binomial coeficient_ which counts the number of ways in which $r$ objects can be chosen from $n$ possibilities when order doesn't matter. In our case, it's the number of ways we can _choose_ exactly 3 of the 5 draws to be even number.

There are $2^5$ potential outcomes, so the cardinality of our state space is: $|\Omega| = 32$.

The ratio of these is the probability of our event: $\mathbb{P}(A) = \frac{|A|}{|\Omega|} = \frac{10}{32} = 0.3125$.

In [9]:
def binomial_coefficient(n: int, r: int) -> int: 
    """
    Return the number of ways in which `r` objects can be chosen 
    from `n` possibilities when order doesn't matter. """
    return int(factorial(n) / factorial(r) / factorial(n - r))

n = 5
r = 3

event_cardinality = binomial_coefficient(n, r)
omega_cardinality = 2**n

print("Probability of event:", event_cardinality / omega_cardinality)

Probability of event: 0.3125


In [10]:
# (Tidy up to remove objects no longer needed.)
del n, r, event_cardinality, omega_cardinality  #, binomial_coefficient

## Question 3
We draw at random 5 numbers from $\left \{1, ..., 100 \right \}$, *without replacement*. What is the
probability that exactly 3 numbers are even?

##### Solution:

As in the previous question, the number of elemental outcomes in our event remains the same: $|A| = 10$. However, because numbers are not replaced back into the set between draws, the probabilities of odds and evens is affected after each draw. 
For example, although the probability of an even number for the first draw will remain at $\frac{50}{100}$, probabilities for the second draw are dependent on the first, e.g. if the first draw is even, then the probability of even in the second draw is $\frac{49}{99}$, whereas the probability of odd in the second draw is $\frac{50}{99}$. There are a different sequence of such fractions for each of the 10 permutations (of 3 even and 2 odd draws), but it can be shown (if one posessed skill and stamina) that the product of all 5 fractions for any of the 10 permutations is equal to $\frac{50 \times 49 \times 48 \times 50 \times 49}{100 \times 99 \times 98 \times 97 \times 96}$. The answer to question is $10 \frac{50 \times 49 \times 48 \times 50 \times 49}{100 \times 99 \times 98 \times 97 \times 96} \approx 0.31891$.

In [11]:
def question_3_type_problem(n_pool: int, n_draw: int, n_success: int) -> float:
    
    assert n_pool % 2 == 0, "The pool must hold an even number of elements."
    assert n_pool / 2 >= n_draw, "Drawing more than half of the pool is not catered for in this implementation."
    
    n_perms = binomial_coefficient(n_draw, n_success)
    
    numerators = np.array(
        [n_pool / 2 - i for i in range(n_success)] +
        [n_pool / 2 - i for i in range(n_draw - n_success)])
    
    denominators = np.array([n_pool - i for i in range(n_draw)])
    
    return n_perms * np.product(numerators) / np.product(denominators)

print("Answer: Probability:", question_3_type_problem(n_pool=100, n_draw=5, n_success=3))

Answer: Probability: 0.318910757055087


Here is an (completely untested) answer for the same problem but for a reduced pool of $\left \{1, ..., 10 \right \}$:

In [12]:
print("For reduced pool of 10: Probability:", question_3_type_problem(n_pool=10, n_draw=5, n_success=3))

For reduced pool of 10: Probability: 0.3968253968253968


## Question 4
Let $Y$ be a number drawn at random from $\left \{ 1, 2, ..., 60 \right \}$; (all numbers are equally
likely to be drawn).

(a) What is the probability that $Y$ is divisible by 2?

(b) What is the probability that $Y$ is divisible by 2 or 3?

(c) What is the probability that $Y$ is divisible by 2 or 3 or 5?


##### Solution:

In [13]:
def divisible_by(n: int, divisors: Iterable) -> bool:
    "Returns whether `n` is divisible by any of `divisors`."
    for e in divisors:
        if n % e == 0:
            return True
    return False

N = 60
omega = set(range(1, N + 1))

print("Probability of a draw being divisable by an element within a given set:")
for divisors in ({2}, {2, 3}, {2, 3, 5}):
    divisables = {e for e in omega if divisible_by(e, divisors)}
    print(divisors, ":", len(divisables) / N)

Probability of a draw being divisable by an element within a given set:
{2} : 0.5
{2, 3} : 0.6666666666666666
{2, 3, 5} : 0.7333333333333333


## Question 5
Let $(a_1, ... , a_n)$ be a random permutation of the integers $ \left \{ 1, 2, ..., n \right \}$ with all permuations equally likely.

   (a) What is the probability that $a_1 = 1$?

   (b) What is the probability that $a_1 = 1$ or $a_2 = 2$?

   (c) What is the probability that $a_1 = 1$ or $a_2 = 2$ or $a_3 = 3$?
   
##### Solution:

The number of permutations in the state space is: $|\Omega| = n!$. 


(a) 

If $a_1$ is fixed, then there remains $(n-1)!$ permutations of the remaining $n-1$ elements.

$$\mathbb{P}(a_1 = 1) = \frac{(n-1)!}{n!} = \frac{1}{n}$$


(b)

$$\mathbb{P}(a_1 = 1 \vee a_2 = 2)$$ 

$$ = \mathbb{P}(a_1 = 1) + \mathbb{P}(a_2 = 2) - \mathbb{P}(a_1 = 1 \wedge a_2 = 2)$$

$$ = \frac{1}{n} + \frac{1}{n} - \frac {\binom{n}{2}}{n^2}$$

$$ = \frac{2}{n} - \frac {n!}{2!n^2(n-2!)} $$



(c)

$$\mathbb{P}(a_1 = 1 \vee a_2 = 2 \vee a_3 = 3)$$ 

$$ = \mathbb{P}(a_1 = 1) + \mathbb{P}(a_2 = 2) + \mathbb{P}(a_3 = 3) - 3 \cdot \mathbb{P}(a_1 = 1 \wedge a_2 = 2) - \mathbb{P}(a_1 = 1 \wedge a_2 = 2 \wedge a_3 = 3)$$

$$ = \frac{3}{n} - 3\frac {\binom{n}{2}}{n^2} - \frac {\binom{n}{3}}{n^3}$$


## Question 6
Testing whether a multivariate polynomial is equal to zero is a standard problem
in computational complexity. We will consider the following simpler setting. Let
$G$ be a polynomial of degree at most $d$ and assume $G$ is not identically zero. Let
$X$ be a number drawn at random (i.e. all numbers are
equally likely to be drawn) 
from $ \left \{ 1, 2, ..., 100d \right \}$. Show that $\mathbb{P} \left (  G(X) \neq 0 \right) \ge 0.99$. (Hint: What does the fundamental theorem of algebra say?)

#### _Solution:_
   
(none)